In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd

# File path
file_path = '/content/Global_pheno.tif'

# Define the bounding box for Australia (approximate)
australia_bbox = {
    "minx": 113.34,  # Minimum longitude
    "maxx": 153.57,  # Maximum longitude
    "miny": -39.14,  # Minimum latitude
    "maxy": -10.68   # Maximum latitude
}

# Convert the bounding box to a GeoJSON format
geometry = [box(australia_bbox["minx"], australia_bbox["miny"], australia_bbox["maxx"], australia_bbox["maxy"])]
geo = gpd.GeoDataFrame({"geometry": geometry}, crs="EPSG:4326")

# Open the .tif file
with rasterio.open(file_path) as src:
    # Reproject the bounding box to match the .tif file's CRS
    geo = geo.to_crs(src.crs)

    # Mask the data using the reprojected bounding box
    out_image, out_transform = mask(src, geo.geometry, crop=True)

    # Get the data array and metadata
    methane_data = out_image[0]
    no_data_value = src.nodata

# Mask the NoData values
methane_data_masked = np.ma.masked_equal(methane_data, no_data_value)

# Check if the data contains only masked values
if np.all(methane_data_masked.mask):
    print("Warning: All data values are masked (NoData) in the selected region.")
else:
    # Normalize data for visualization
    methane_data_normalized = (methane_data_masked - np.min(methane_data_masked)) / (np.max(methane_data_masked) - np.min(methane_data_masked))

    # Plot the data
    plt.figure(figsize=(10, 8))
    plt.imshow(methane_data_normalized, cmap='plasma', interpolation='none')  # Changed color map to 'plasma' for better visibility
    plt.colorbar(label='Normalized Methane Concentration')
    plt.title('Methane Concentrations over Australia (2023)')
    plt.grid(True)
    plt.show()


In [ ]:
!pip install rasterio geopandas matplotlib

In [ ]:
import rasterio
import matplotlib.pyplot as plt

# Path to the uploaded .tif file
tif_path = '/content/Australia_Fire_2014_2023.tif'  # Replace with your file name if different

# Open the .tif file
with rasterio.open(tif_path) as dataset:
    # Read the data (assuming single-band raster)
    fire_data = dataset.read(1)  # First band

    # Plot the data
    plt.figure(figsize=(10, 8))
    plt.imshow(fire_data, cmap='hot', extent=dataset.bounds)
    plt.colorbar(label='Fire Intensity')
    plt.title('Fire Intensity in Australia (2014-2023)')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.show()
